### Getting the files & import statements ###

In [12]:
import pandas as pd
import numpy as np
import os

In [21]:
# Get participant number 
participant_number = input("Enter participant number: ")

base_path = "/Users/alina/Downloads/Thesis/raw/"

# Folder path for the participant
folder_path = os.path.join(base_path, f"Participant {participant_number}")

# Get recording number 
recording_number = input("Enter recording number: ")

# Recording folder path
recording_path = os.path.join(folder_path, f"P{participant_number}_{recording_number}")

# Get files
world_timestamps_file = os.path.join(recording_path, "world_timestamps.csv")
detections_file = os.path.join(recording_path, f"detections_{participant_number}_{recording_number}.csv")
fixations_file = os.path.join(recording_path, "fixations.csv")
face_file = os.path.join(recording_path, "fixations_on_face.csv")

# Load as dfs
world_timestamps_df = pd.read_csv(world_timestamps_file) #contains timestamps for each video frame
detections_df = pd.read_csv(detections_file) #contains detections from YOLO
fixations_df = pd.read_csv(fixations_file) #fixations data
face_df = pd.read_csv(face_file) #fixations on face data from FaceMapper

### Matching video frames and timestamps

In [22]:
## function to see if the gaze is inside the bounding box of an object
def is_gaze_in_bbox(gaze_x, gaze_y, x_min, y_min, x_max, y_max):
    return x_min <= gaze_x <= x_max and y_min <= gaze_y <= y_max

## function to prioritise the smallest object fixated
def filter_objects_by_area(group):
    group = group.copy()
    group["area"] = (group["X_max"] - group["X_min"]) * (group["Y_max"] - group["Y_min"])

    # Find the minimum area
    min_area = group["area"].min()
    
    # Get all objects with the minimum area
    smallest_objects = group[group["area"] == min_area]
    
    if len(smallest_objects) == 1:
        return smallest_objects
    else:
        person_objects = smallest_objects[smallest_objects["Class"] == "person"]
        if not person_objects.empty:
            # Return only one person object (the first one)
            return person_objects.iloc[[0]]

    # If no person among smallest, return just one of the smallest objects
    return smallest_objects.iloc[[0]]

In [23]:
## world_timestamps.csv file from pupil cloud represents timestamps for each video frame
# merge_asof is like a nearest match join for ordered time-series data

# renaming so all timestamp columns are named the same to merge the dfs
fixations_df["timestamp [ns]"] = fixations_df["start timestamp [ns]"]

## making a video_df which contains matching frames and ts
ts = world_timestamps_df['timestamp [ns]']

video_df = pd.DataFrame({
        "Frame": np.arange(len(world_timestamps_df)),
        "timestamp [ns]": ts,
    })

# Make sure everything is sorted by timestamp so merge_asof works properly
fixations_df = fixations_df.sort_values("start timestamp [ns]")
world_timestamps_df = world_timestamps_df.sort_values("timestamp [ns]")

# Merge-asof to assign the closest frame_id to each fixation
fixations_with_frame = pd.merge_asof(
    fixations_df,
    video_df,
    on="timestamp [ns]",
    direction="nearest",  # to match the closest timestamp from video_df, whether it's before or after the fixation timestamp
) 

# merging to assign fixations to frames
fixations_with_detections = pd.merge(
    fixations_with_frame,
    detections_df,
    on="Frame",
    how="left" # keep all the fixations, even if a frame has no detection => nan
)

# calculate in_bbox 
fixations_with_detections["in_bbox"] = fixations_with_detections.apply(
    lambda row: is_gaze_in_bbox(
        row["fixation x [px]"], row["fixation y [px]"],
        row["X_min"], row["Y_min"], row["X_max"], row["Y_max"]
    ),
    axis=1
)

# filter only fixations that match at least one bounding box
in_bbox_fixations = fixations_with_detections[fixations_with_detections["in_bbox"] == True]

#  group by fixation ID and drop "person" if there is another class in same fixation group
filtered_fixations = in_bbox_fixations.groupby(
    "Frame", group_keys=False # grouping on frame as i need fixation id column later
).apply(filter_objects_by_area, include_groups=False)

fixations_on_persons = filtered_fixations.loc[filtered_fixations["Class"] == "person"]
fixations_on_persons.head()

,section id,recording id,fixation id,start timestamp [ns],end timestamp [ns],duration [ms],fixation x [px],fixation y [px],azimuth [deg],elevation [deg],timestamp [ns],X_min,Y_min,X_max,Y_max,Confidence,Class,in_bbox,area
7,993f45e1-9303-4f5b-9c0e-1fee427d17cd,0f400786-0ce5-43ca-984a-14062fc58ef7,3,1749034195711882116,1749034196052132116,340,1058.061,286.346,15.821936,20.267647,1749034195711882116,807.142944,0.000000,1479.967651,1200.000000,0.911381,person,True,807389.648438
13,993f45e1-9303-4f5b-9c0e-1fee427d17cd,0f400786-0ce5-43ca-984a-14062fc58ef7,4,1749034196087257116,1749034196662760116,576,1044.274,263.903,14.997352,21.712247,1749034196087257116,828.085938,0.000000,1463.281250,1200.000000,0.892459,person,True,762234.375000
18,993f45e1-9303-4f5b-9c0e-1fee427d17cd,0f400786-0ce5-43ca-984a-14062fc58ef7,5,1749034196727756116,1749034197008008116,280,908.057,438.251,5.632528,10.813404,1749034196727756116,823.200073,15.175247,1456.933105,1199.298096,0.915172,person,True,750417.763315
24,993f45e1-9303-4f5b-9c0e-1fee427d17cd,0f400786-0ce5-43ca-984a-14062fc58ef7,6,1749034197073130116,1749034197288265116,215,1133.253,280.604,20.865379,20.436278,1749034197073130116,828.218994,7.935944,1463.416504,1199.105469,0.900965,person,True,756627.916082
29,993f45e1-9303-4f5b-9c0e-1fee427d17cd,0f400786-0ce5-43ca-984a-14062fc58ef7,7,1749034197438505116,1749034197558508116,120,694.005,548.058,-8.186850,3.771427,1749034197438505116,407.560272,0.000000,884.521790,1197.571533,0.924496,person,True,571195.535593


### Creating a new file with categorised fixations

In [24]:
# filtering to only fixations on bodies
fixations_on_persons = fixations_on_persons[fixations_on_persons["in_bbox"] == True]

# initialising a df for all types of fixations
categorized_fixations_all = []

# using FaceMapper file to get all the basic fixations information
for _, row in face_df.iterrows():
    timestamp = row['start timestamp [ns]']
    fixation_id = row['fixation id']
    fix_duration = (row['end timestamp [ns]'] - row['start timestamp [ns]'])/1000000     #converting to ms
    fixation_type = "background" #make default type

    # face fixations
    if row.get('fixation on face') == True:
        fixation_type = "face"

    # body fixations (only if not already labeled as face)
    elif not fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].empty:
        fixation_type = "body"
        matching_row = fixations_on_persons[fixations_on_persons['timestamp [ns]'] == timestamp].iloc[0]
        fixation_id = matching_row['fixation id']
        fix_duration = matching_row['duration [ms]']

    categorized_fixations_all.append({
        'timestamp [ns]': timestamp,
        'type': fixation_type,
        'fixation id': fixation_id,
        'duration [ms]': fix_duration
    })

# saving a file with fixations categorised by face, body and background
all_categorized_fixations_df = pd.DataFrame(categorized_fixations_all)
saving_path = os.path.join(recording_path, f"fixations_on_everything_{participant_number}_{recording_number}.csv")
all_categorized_fixations_df.to_csv(saving_path, index=False)

all_categorized_fixations_df.head()

,timestamp [ns],type,fixation id,duration [ms]
0,1749034195241383116,background,1,115.125
1,1749034195386508116,background,2,225.249
2,1749034195711882116,face,3,340.250
3,1749034196087257116,face,4,575.503
4,1749034196727756116,body,5,280.000
